In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import seaborn as sns
import time
import tensorflow as tf


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, BatchNormalization, Input

In [2]:
df_no_anomaly = pd.read_parquet("df_no_anomaly.parquet.gzip")

In [3]:
print(len(df_no_anomaly))
df_no_anomaly = pd.pivot_table(df_no_anomaly,values='diff_square',index='Time')
df_no_anomaly.head()

600001


,diff_square
Time,
2021-04-13 15:15:00.000,302760000.0
2021-04-13 15:15:00.002,302760000.0
2021-04-13 15:15:00.004,302760000.0
2021-04-13 15:15:00.006,302760000.0
2021-04-13 15:15:00.008,302760000.0


In [4]:
# Normalize and save the mean and std we get,
# for normalizing test data.
training_mean = df_no_anomaly.mean()
training_std = df_no_anomaly.std()
df_training_value = (df_no_anomaly - training_mean) / training_std
print("Number of training samples:", len(df_training_value))

Number of training samples: 600001


In [5]:
df_training_value.values

array([[-0.4654512],
       [-0.4654512],
       [-0.4654512],
       ...,
       [ 0.6428225],
       [ 0.6428225],
       [ 0.6428225]])

In [26]:
dataset = tf.data.Dataset.from_tensor_slices(df_training_value.values)

In [33]:
dataset_win = dataset.window(100)

In [34]:
for window in dataset_win:
    print(len(window))
    break

100


In [28]:
next(iter(dataset))

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-0.4654512])>

In [38]:
next(iter(dataset_win))

<_VariantDataset shapes: (1,), types: tf.float64>

In [9]:
# iterator = iter(dataset)
# iterator

In [10]:
# iterator.element_spec

In [11]:
# print(iterator.get_next())

In [12]:
# for example_inputs in dataset.take(1):
#   print(f'Inputs shape (batch, time, features): {example_inputs.shape}')

In [13]:
# TIME_STEPS = 100
# # Generated training sequences for use in the model.
# def create_sequences(values, time_steps=TIME_STEPS):
#     for i in range(len(values) - time_steps):
#         yield values[i : (i + time_steps)]


# def data_gen(Values):
#     X_train_c = []
#     for x in create_sequences(Values):
#         X_train_c.append(x)  
#     return np.asarray(X_train_c)

# x_train = data_gen(df_training_value.values)
# print("Training input shape: ", x_train.shape)

In [43]:

# model = keras.Sequential(
#     [
#         layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
#         layers.Conv1D(
#             filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
#         ),
#         layers.Dropout(rate=0.2),
#         layers.Conv1D(
#             filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
#         ),
#         layers.Conv1DTranspose(
#             filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
#         ),
#         layers.Dropout(rate=0.2),
#         layers.Conv1DTranspose(
#             filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
#         ),
#         layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
#     ]
# )
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss="mse")
# model.summary()

# model = Sequential()
# model.add(LSTM(32, input_shape=(x_train.shape[1], x_train.shape[2])))
# model.add(Dropout(rate=0.2))
# model.add(RepeatVector(x_train.shape[1]))
# model.add(LSTM(32, return_sequences=True))
# model.add(Dropout(rate=0.2))
# model.add(TimeDistributed(Dense(x_train.shape[2])))
# model.compile(optimizer='adam', loss='mae')
# model.summary()



input_window = Input(shape=(100,))

x = Dense(32, activation='relu')(input_window)
x = BatchNormalization()(x)
x = Dense(16, activation='relu')(x)
x = BatchNormalization()(x)
encoded = Dense(8, activation='relu')(x)

# "decoded" is the lossy reconstruction of the input
x = Dense(16, activation='relu')(encoded)
x = BatchNormalization()(x)
x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)
decoded = Dense(100, activation='sigmoid')(x)

# this model maps an input to its reconstruction
model = Model(input_window, decoded)

# this model maps an input to its encoded representation
# encoder = Model(input_window, encoded)

model.summary()

model.compile(optimizer='adam', loss='mae')


Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                3232      
_________________________________________________________________
batch_normalization_12 (Batc (None, 32)                128       
_________________________________________________________________
dense_19 (Dense)             (None, 16)                528       
_________________________________________________________________
batch_normalization_13 (Batc (None, 16)                64        
_________________________________________________________________
dense_20 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_21 (Dense)             (None, 16)               

In [45]:
# history = model.fit(
#     x_train,
#     x_train,
#     epochs=1,
#     batch_size=3072,
#     validation_split=0.2,
#     callbacks=[
#         keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, mode="min")
#     ],
# )

history = model.fit(dataset, epochs=1)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:789 run_step  **
        outputs = model.t

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
# Get train MAE loss.
x_train_pred = model.predict(x_train)



In [ ]:
x_train_pred.shape

In [ ]:
x_train = x_train.reshape(x_train_pred.shape[0],x_train_pred.shape[1])

In [ ]:
train_mae_loss = np.mean(np.abs(x_train_pred - x_train), axis=1)

plt.hist(train_mae_loss, bins=50)
plt.xlabel("Train MAE loss")
plt.ylabel("No of samples")
plt.show()

# Get reconstruction loss threshold.
threshold = np.max(train_mae_loss)
print("Reconstruction error threshold: ", threshold)

In [ ]:
# Checking how the first sequence is learnt
plt.plot(x_train[1], label="x_train")
plt.plot(x_train_pred[1],label= "x_train_pred")
plt.legend()
plt.show()

In [ ]:
df_test_value = (df_with_anomaly - training_mean) / training_std
fig, ax = plt.subplots()
df_test_value.plot(legend=False, ax=ax)
plt.show()

# Create sequences from test values.
x_test = data_gen(df_test_value.values)
print("Test input shape: ", x_test.shape)

# Get test MAE loss.
x_test_pred = model.predict(x_test)

x_test = x_test.reshape(x_test_pred.shape[0],x_test_pred.shape[1])

test_mae_loss = np.mean(np.abs(x_test_pred - x_test), axis=1)
test_mae_loss = test_mae_loss.reshape((-1))

plt.hist(test_mae_loss, bins=50)
plt.xlabel("test MAE loss")
plt.ylabel("No of samples")
plt.show()

# Detect all the samples which are anomalies.
anomalies = test_mae_loss > threshold
print("Number of anomaly samples: ", np.sum(anomalies))
print("Indices of anomaly samples: ", np.where(anomalies))

In [ ]:
# data i is an anomaly if samples [(i - timesteps + 1) to (i)] are anomalies
anomalous_data_indices = []
for data_idx in range(TIME_STEPS - 1, len(df_test_value) - TIME_STEPS + 1):
    if np.all(anomalies[data_idx - TIME_STEPS + 1 : data_idx]):
        anomalous_data_indices.append(data_idx)

In [ ]:
if len(anomalous_data_indices):
    df_subset = df_with_anomaly.iloc[anomalous_data_indices]
    fig, ax = plt.subplots()
    df_with_anomaly.plot(legend=False, ax=ax)
    df_subset.plot(legend=False, ax=ax, color="r")
    plt.show()
else:
    print("No Anomalies found!")

In [ ]:
plt.plot(x_train[0])
plt.plot(x_train_pred[0])
plt.show()